<a href="https://colab.research.google.com/github/RicardoCastelhano/image-based-product-recommendation/blob/main/image_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sistema de Recomendação por Similaridade Visual de Imagens
Objetivo

Desenvolver um sistema de recomendação capaz de indicar produtos visualmente semelhantes a partir de uma imagem de entrada, utilizando Deep Learning e Visão Computacional, sem uso de informações textuais

#Módulo 1 — Configuração do Ambiente

Instalação das bibliotecas necessárias para execução do projeto no Google Colab.

In [1]:
!pip install -q kaggle tensorflow scikit-learn matplotlib pillow


#Módulo 2 — Download do Dataset via API do Kaggle

Para a obtenção do dataset utilizado neste projeto, foi empregada a API oficial do Kaggle, que permite o acesso autenticado a bases de dados públicas diretamente por código. Essa abordagem é amplamente utilizada em projetos de Ciência de Dados, pois garante reprodutibilidade, automação e facilidade de execução em ambientes como o Google Colab.

Mesmo sendo um dataset público, o Kaggle exige autenticação para realizar downloads programáticos. Para isso, é necessário gerar um token de acesso, disponibilizado no arquivo kaggle.json, que funciona como uma credencial segura para autorização do download.

🔑 Passo a passo do processo:

O usuário deve acessar sua conta no Kaggle e, na seção Account, gerar um token da API clicando em Create New API Token.

Esse processo gera o arquivo kaggle.json, que contém as credenciais necessárias para autenticação.

O arquivo é então enviado para o ambiente do Google Colab.

Após o upload, o token é configurado no diretório padrão esperado pela API do Kaggle.

Com a autenticação concluída, o dataset Fashion Product Images Dataset é baixado automaticamente via linha de comando e descompactado para uso no projeto.

Essa estratégia elimina a necessidade de downloads manuais, torna o projeto mais organizado e permite que qualquer pessoa execute o notebook do zero, desde que possua uma conta no Kaggle.

⚠️ Observação: o arquivo kaggle.json contém informações sensíveis e não deve ser versionado ou enviado para repositórios públicos, como o GitHub.

In [ ]:
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets list | head

!kaggle datasets download -d paramaggarwal/fashion-product-images-dataset

!ls -lh | grep fashion


!unzip fashion-product-images-dataset.zip -d dataset

import os

if os.path.exists("dataset/images"):
    print("✅ Dataset baixado e extraído com sucesso!")
    print(f"📸 Total de imagens: {len(os.listdir('dataset/images'))}")
else:
    print("❌ Dataset não encontrado. Verifique as etapas anteriores.")





Saving kaggle.json to kaggle (1).json
ref                                                              title                                                  size  lastUpdated                 downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------  -----------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
neurocipher/heartdisease                                         Heart Disease                                          3491  2025-12-11 15:29:14.327000           2114         95  1.0              
neurocipher/student-performance                                  Student Performance                                   49705  2025-12-12 12:06:28.973000           1261         70  1.0              
wardabilal/spotify-global-music-dataset-20092025                 Spotify Global Music Dataset (2009–2025)            1289021  2025-11-11 09:43:05.933000          16387   

#Módulo 3 — Importação de Bibliotecas

Carregamento das bibliotecas utilizadas no pré-processamento, modelagem e visualização.

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from sklearn.metrics.pairwise import cosine_similarity


#Módulo 4 — Pré-processamento das Imagens

Padronização das imagens para o formato esperado pela CNN

In [4]:
IMG_SIZE = (224, 224)
IMAGE_FOLDER = "dataset/images"

def load_and_preprocess(img_path):
    img = Image.open(img_path).convert("RGB")
    img = img.resize(IMG_SIZE)
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array)


#Módulo 5 — Modelo de Deep Learning

Uso de uma CNN pré-treinada (ResNet50) como extratora de características visuais.

In [5]:
model = ResNet50(
    weights="imagenet",
    include_top=False,
    pooling="avg"
)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Módulo 6 — Geração dos Embeddings

Conversão das imagens em vetores numéricos que representam suas características visuais.

In [6]:
image_paths = [
    os.path.join(IMAGE_FOLDER, img)
    for img in os.listdir(IMAGE_FOLDER)[:2000]
]

embeddings = []

for path in image_paths:
    img = load_and_preprocess(path)
    emb = model.predict(img, verbose=0)
    embeddings.append(emb[0])

embeddings = np.array(embeddings)


FileNotFoundError: [Errno 2] No such file or directory: 'dataset/images'

#Módulo 7 — Sistema de Recomendação

Cálculo da similaridade entre imagens utilizando similaridade do cosseno.

In [ ]:
def recommend(img_path, top_n=5):
    query = load_and_preprocess(img_path)
    query_emb = model.predict(query, verbose=0)

    sims = cosine_similarity(query_emb, embeddings)[0]
    idxs = np.argsort(sims)[-top_n-1:-1][::-1]

    plt.figure(figsize=(15,5))
    plt.subplot(1, top_n+1, 1)
    plt.imshow(Image.open(img_path))
    plt.title("Imagem de Consulta")
    plt.axis("off")

    for i, idx in enumerate(idxs):
        plt.subplot(1, top_n+1, i+2)
        plt.imshow(Image.open(image_paths[idx]))
        plt.title(f"Similaridade: {sims[idx]:.2f}")
        plt.axis("off")

    plt.show()


#Módulo 8 — Teste do Sistema

Execução do sistema de recomendação a partir de uma imagem de entrada.

In [ ]:
test_image = image_paths[10]
recommend(test_image)
